In [1]:
import pandas as pd
import numpy as np
df=pd.read_csv("/home/xinyang/Kaggle/TRD/TRD_Dalyr.csv",encoding='utf-16',sep='\t')
df2=pd.read_csv("/home/xinyang/Kaggle/TRD/TRD_Dalyr1.csv",encoding='utf-16',sep='\t')
df_con1=pd.concat([df,df2])
df4=pd.read_csv("/home/xinyang/Kaggle/TRD2/TRD_Dalyr.csv",encoding='utf-16',sep='\t')
df5=pd.read_csv("/home/xinyang/Kaggle/TRD2/TRD_Dalyr1.csv",encoding='utf-16',sep='\t')
df6=pd.read_csv("/home/xinyang/Kaggle/TRD2/TRD_Dalyr2.csv",encoding='utf-16',sep='\t')
df_con2=pd.concat([df4,df5,df6])
df7=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr.csv",encoding='utf-16',sep='\t')
df8=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr1.csv",encoding='utf-16',sep='\t')
df9=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr2.csv",encoding='utf-16',sep='\t')
df10=pd.read_csv("/home/xinyang/Kaggle/TRD3/TRD_Dalyr3.csv",encoding='utf-16',sep='\t')
df_con3=pd.concat([df7,df8,df9,df10])
df_con=pd.concat([df_con1,df_con2,df_con3])

/home/xinyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  """
/home/xinyang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [2]:
df3=pd.read_csv("/home/xinyang/Kaggle/zcfg/FS_Combas.csv",encoding='utf-16',sep='\t')
df3=df3.rename(columns={'Accper':'Trddt'})
df3['islong'] = 1

In [3]:
# Specify the Stkcd number. Use 1 as an example here. 
df3_1 = df3[df3.Stkcd==1]
df_con_1 = df_con[df_con.Stkcd == 1]
df_merge_1 = df3_1.merge(df_con_1,on=['Stkcd','Trddt'],how='outer')

In [4]:
df_merge_1.sort_values('Trddt',inplace=True)
df_merge_1 = df_merge_1.reset_index()
df_merge_1 = df_merge_1.drop('index',1)

In [5]:
a = df_merge_1.loc[0,'Typrep':'islong']
for i in range(3465):
    if df_merge_1.loc[i,'islong'] == 1:
        a = df_merge_1.loc[i,'Typrep':'islong']
    else:
        df_merge_1.loc[i,'Typrep':'islong'] = a

In [6]:
df_merge_1.dropna(axis=1,how='all')
df_merge_1 = df_merge_1.fillna(0)
df_merge_1 = df_merge_1.loc[:, (df_merge_1 != 0).any(axis=0)]

In [7]:
df_merge_1 = df_merge_1.drop(['Trddt','Typrep','Capchgdt'],axis =1)

In [8]:
# Append the dataframe that 10-day's data is merged to one row
num = df_merge_1.shape[1]-1
for i in range(9):
    columns = []
    for j in range(num):
        columns.append(str(i)+'_'+str(j))
    df = df_merge_1.iloc[:(-i-1):,1:(num+1)].append(df_merge_1.iloc[:(i+1),1:(num+1)])
    df.columns= columns
    df = df.reset_index(drop=True)
    df_merge_1 = pd.concat([df_merge_1,df],axis=1)   

In [9]:
df_merge_1.shape

(3465, 791)

In [10]:
df_merge_1.head()

,Stkcd,A0b1103000,A0b1104000,A0b1105000,A0f1106000,A001107000,A0f1108000,A001109000,A001111000,A001112000,...,8_69,8_70,8_71,8_72,8_73,8_74,8_75,8_76,8_77,8_78
0,1,1.910729e+10,5.236464e+09,0.0,3.023113e+09,0.0,0.0,1.104687e+10,-717910679.0,52785247.0,...,0.0,0.000000,0.000000,0.00,0.00,0.00,0.00,0.0,0.00,0.0
1,1,1.910729e+10,5.236464e+09,0.0,3.023113e+09,0.0,0.0,1.104687e+10,-717910679.0,52785247.0,...,11465602.0,-0.010622,-0.010622,9189040.01,12686760.41,6.59,6.46,4.0,6.59,1.0
2,1,1.910729e+10,5.236464e+09,0.0,3.023113e+09,0.0,0.0,1.104687e+10,-717910679.0,52785247.0,...,20718558.0,-0.009202,-0.009202,9104478.29,12570011.08,6.55,6.35,4.0,6.52,1.0
3,1,1.910729e+10,5.236464e+09,0.0,3.023113e+09,0.0,0.0,1.104687e+10,-717910679.0,52785247.0,...,17333840.0,0.009288,0.009288,9189040.01,12686760.41,6.59,6.45,4.0,6.50,1.0
4,1,1.910729e+10,5.236464e+09,0.0,3.023113e+09,0.0,0.0,1.104687e+10,-717910679.0,52785247.0,...,12302853.0,-0.001534,-0.001534,9174946.39,12667302.19,6.60,6.46,4.0,6.58,1.0


In [27]:
df = df_merge_1.loc[:,'Dretnd']
df.iloc[:(-11)].append(df.iloc[:11])
df = df.reset_index(drop=True)
df_merge_1['goal'] = df

In [32]:
import featuretools as ft
# Create es
es = ft.EntitySet(id="rigegu")
df_first = df_merge_1.loc[:,'A0b1103000':'Trdsta']
df_first = df_first.reset_index()
es =  es.entity_from_dataframe(entity_id="first",dataframe=df_first,index = "index")

In [35]:
df_second = df_merge_1.loc[:,'0_0':'0_78']
df_second = df_second.reset_index()
es =  es.entity_from_dataframe(entity_id="second",dataframe=df_second,index = "index")
df_third = df_merge_1.loc[:,'1_0':'1_78']
df_third = df_third.reset_index()
es =  es.entity_from_dataframe(entity_id="third",dataframe=df_third,index = "index")
df_fourth = df_merge_1.loc[:,'2_0':'2_78']
df_fourth = df_fourth.reset_index()
es =  es.entity_from_dataframe(entity_id="fourth",dataframe=df_fourth,index = "index")
df_fifth = df_merge_1.loc[:,'3_0':'3_78']
df_fifth = df_fifth.reset_index()
es =  es.entity_from_dataframe(entity_id="fifth",dataframe=df_fifth,index = "index")
df_sixth = df_merge_1.loc[:,'4_0':'4_78']
df_sixth = df_sixth.reset_index()
es =  es.entity_from_dataframe(entity_id="sixth",dataframe=df_sixth,index = "index")
df_seventh = df_merge_1.loc[:,'5_0':'5_78']
df_seventh = df_seventh.reset_index()
es =  es.entity_from_dataframe(entity_id="seventh",dataframe=df_seventh,index = "index")
df_eighth = df_merge_1.loc[:,'6_0':'6_78']
df_eighth = df_eighth.reset_index()
es =  es.entity_from_dataframe(entity_id="eighth",dataframe=df_eighth,index = "index")
df_ninth = df_merge_1.loc[:,'7_0':'7_78']
df_ninth = df_ninth.reset_index()
es =  es.entity_from_dataframe(entity_id="ninth",dataframe=df_ninth,index = "index")
df_tenth = df_merge_1.loc[:,'8_0':'8_78']
df_tenth = df_tenth.reset_index()
es =  es.entity_from_dataframe(entity_id="tenth",dataframe=df_tenth,index = "index")
df_goal = df_merge_1['goal']
df_goal = df_goal.reset_index()
es =  es.entity_from_dataframe(entity_id="goal",dataframe=df_goal,index = "index")

In [36]:
 feature_matrix, feature_defs = ft.dfs(entityset=es, 
                                       target_entity="goal",
                                       agg_primitives=["max","median","time_since_last","min","avg_time_between",
                                                      "all","mode","n_most_common","any","mean","time_since_first"
                                                      ,"percent_true","last","sum","num_true","trend","std","count"
                                                      ,"num_unique","skew"],
                                      max_depth = 3)

In [38]:
es

Entityset: rigegu
  Entities:
    first [Rows: 3465, Columns: 80]
    second [Rows: 3465, Columns: 80]
    third [Rows: 3465, Columns: 80]
    fourth [Rows: 3465, Columns: 80]
    fifth [Rows: 3465, Columns: 80]
    sixth [Rows: 3465, Columns: 80]
    seventh [Rows: 3465, Columns: 80]
    eighth [Rows: 3465, Columns: 80]
    ninth [Rows: 3465, Columns: 80]
    tenth [Rows: 3465, Columns: 80]
    goal [Rows: 3465, Columns: 2]
  Relationships:
    No relationships